In [5]:
!pip install num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 14.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0bcb7cd06caa928388b905c6edcd853ba24542a55b6701363544c751a950ad57
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
from multiprocessing import Process, Queue
from num2words import num2words
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import json
import multiprocessing
import nltk
import os
import os, random, string, re
import pandas as pd
import pickle
import random
import shutil
import sys
import time
import torch.multiprocessing

In [9]:
# copy datsets tar.gz files from https://zenodo.org/records/10042019 under /content/drive/MyDrive/NLP/original-tables/
# prompt: create folders under /content/drive/MyDrive/NLP/original-tables/ called tus-santos, wiki-jaccard, wiki-containment


folders_to_create = ["tus-santos", "wiki-jaccard", "wiki-containment"]
base_path = "/content/drive/MyDrive/NLP/original-tables/"

for folder in folders_to_create:
  path = os.path.join(base_path, folder)
  if not os.path.exists(path):
    os.makedirs(path)
    print(f"Folder '{folder}' created successfully at {path}")
  else:
    print(f"Folder '{folder}' already exists at {path}")

Folder 'tus-santos' created successfully at /content/drive/MyDrive/NLP/original-tables/tus-santos
Folder 'wiki-jaccard' created successfully at /content/drive/MyDrive/NLP/original-tables/wiki-jaccard
Folder 'wiki-containment' created successfully at /content/drive/MyDrive/NLP/original-tables/wiki-containment


In [ ]:
# prompt: unzip tar file
!tar -xvf /content/drive/MyDrive/NLP/original-tables/tus-santos.tar.bz2 -C /content/drive/MyDrive/NLP/original-tables/tus-santos
!tar -xvf /content/drive/MyDrive/NLP/original-tables/wiki-jaccard.tar.bz2 -C /content/drive/MyDrive/NLP/original-tables/wiki-jaccard
!tar -xvf /content/drive/MyDrive/NLP/original-tables/wiki-containment.tar.bz2 -C /content/drive/MyDrive/NLP/original-tables/wiki-containment

In [12]:
nltk.download('punkt_tab')
cpu_cores = [ i for i in range(70)]

os.sched_setaffinity(os.getpid(), cpu_cores)

torch.multiprocessing.set_sharing_strategy('file_system')

def analyze_column_values(df, column_name):
    value_counts = df[column_name].astype(str).value_counts()

    sorted_values = value_counts.index.tolist()

    n = len(sorted_values)
    col = ', '.join(sorted_values)

    lengths = [len(str(value)) for value in sorted_values]
    max_len = max(lengths)
    min_len = min(lengths)
    avg_len = sum(lengths) / len(lengths)
    tokens = f"{column_name} contains {str(n)} values ({str(max_len)}, {str(min_len)}, {str(avg_len)}): {col}"

    tokens = nltk.word_tokenize(tokens)
    truncated_tokens = tokens[:512]
    truncated_sentence = ' '.join(truncated_tokens)
    return truncated_sentence

def evaluate4(df):
    columns = df.columns.tolist()
    sentens_list = []
    for column in columns:
        s = analyze_column_values(df,column)
        sentens_list.append(s)
    return sentens_list

def get_file_columns(file_path):
    df = pd.read_csv(file_path,engine='python',nrows =1)
    columns = len(df.columns)
    return columns

def partition_files(file_paths, m):
    random.shuffle(file_paths)
    file_info = []

    current_group = []
    current_columns = 0
    stime = time.time()
    for file_path in file_paths:
        columns = get_file_columns(file_path)
        if current_columns + columns > m:
            file_info.append(current_group)
            current_group = [file_path + "_" + str(columns)]
            current_columns = columns
        else:
            current_group.append(file_path+ "_" + str(columns))
            current_columns += columns
    endtime = time.time()
    if current_group:
        file_info.append(current_group)
    return file_info

def create_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        print("Folder and its content deleted.")

    os.makedirs(path)
    print("Folder created.")

def read_pkl_files(folder_path):
    file_list = os.listdir(folder_path)

    re_dict = {}

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)

        if file_name.endswith(".pkl"):
            try:
                with open(file_path, 'rb') as file:
                    obj = pickle.load(file)

                re_dict.update(obj)
            except Exception as e:
                print("Error occurred while reading", file_name, ":", str(e))
    return re_dict

def process_task4(i,input_values,queue,queue_inforgather,file_dic_path):

    dict = {}
    for input_value in input_values:
        k = struct_dic_key(input_value)
        try:
            df = pd.read_csv(input_value, low_memory=False)
        except Exception as e:
            print("error filename:",input_value)
            continue
        embdings = evaluate4(df)
        dict[k] = embdings
        queue.put(1)
    filename = os.path.join(file_dic_path,str(i)+".pkl")
    with open(filename, 'wb') as file:
        pickle.dump(dict, file)
    queue.put((-1, "test-pid"))

def struct_dic_key(filepath):
    elelist = filepath.split(os.sep)
    return elelist[-2] + "-" + elelist[-1]

def split_list(lst, num_parts):
    avg = len(lst) // num_parts
    remainder = len(lst) % num_parts

    result = []
    start = 0
    for i in range(num_parts):
        if i < remainder:
            end = start + avg + 1
        else:
            end = start + avg
        result.append(lst[start:end])
        start = end

    return result

def process_table_sentense(filepathstore,datadir,data_pkl_name,tmppath= "/data/lijiajun/webtable_tmp",split_num=10):
    list_of_tuples_name = data_pkl_name
    dir = datadir
    file_dic_path = tmppath

    os.makedirs(filepathstore,exist_ok=True)
    create_folder(file_dic_path)

    filelist = []

    for root, dirs, files in os.walk(dir):
        for file in files:
            if file == 'small_join.csv' or file  == 'large_join.csv':
                continue
            filepath = os.path.join(root, file)
            if os.path.isfile(filepath):
                filelist.append(filepath)

            else:
                print(f"file: {filepath} is not a file ,pass")
    print(f"split1 all file ,filelistlen: {len(filelist)} added to filelist")


    inputs = filelist
    sub_file_ls = split_list(inputs, split_num)
    process_list = []

    queues = [Queue() for i in range(split_num)]
    finished = [False for i in range(split_num)]

    bars = [tqdm(total=len(sub_file_ls[i]), desc=f"bar-{i}", position=i) for i in range(split_num)]
    results = [None for i in range(split_num)]
    queue_inforgather = multiprocessing.Manager().Queue()

    for i in range(split_num):
        process = Process(target=process_task4, args=(i,sub_file_ls[i], queues[i], queue_inforgather,file_dic_path))
        process_list.append(process)
        process.start()

    while True:
        for i in range(split_num):
            queue = queues[i]
            bar = bars[i]
            try:
                res = queue.get_nowait()
                if isinstance(res, tuple) and res[0] == -1:
                    finished[i] = True
                    results[i] = res[1]
                    continue
                bar.update(res)
            except Exception as e:
                continue
        if all(finished):
            break

    for process in process_list:
        process.join()

    result_dict = read_pkl_files(file_dic_path)

    list_of_tuples = list(result_dict.items())

    with open(os.path.join(filepathstore,list_of_tuples_name),'wb') as file:
        pickle.dump(list_of_tuples,file)
    print("pickle sucesss")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [14]:
# ── CONFIGURATION ────────────────────────────────────────────────────────────
DATA_DIR    = "/content/drive/MyDrive/NLP/original-tables/wiki-jaccard/tables"
LABELS_JSON = "/content/drive/MyDrive/NLP/original-tables/wiki-jaccard/labels.json"
MODEL_NAME  = "all-mpnet-base-v2"
LOCAL_OUT   = "/content/join_scores_wiki_jaccard.csv"   # write here first
DRIVE_OUT   = "/content/drive/MyDrive/NLP/join-scores/wiki-jaccard.csv"
BATCH_SIZE  = 128  # increase if GPU memory allows
# ───────────────────────────────────────────────────────────────────────────────

def load_pairs(json_path):
    with open(json_path) as f:
        data = json.load(f)["train"]
    # extract unique (table1, table2) filename pairs
    return list({(r["table1"]["filename"], r["table2"]["filename"]) for r in data})

def embed_table(path, model, device):
    """Read a CSV once, build all column-texts, batch-encode, and return (cols, embs)."""
    df = pd.read_csv(path, dtype=str)  # single read
    cols = df.columns.tolist()
    texts = [analyze_column_values(df, c) for c in cols]
    del df  # free memory
    embs = model.encode(
        texts,
        convert_to_tensor=True,
        device=device,
        batch_size=BATCH_SIZE,
        show_progress_bar=False
    )
    return cols, embs

def main():
    # 1) Load table-pairs from labels.json
    pairs = load_pairs(LABELS_JSON)
    tables = sorted({t for pair in pairs for t in pair})
    print(f"Will process {len(tables)} distinct tables, {len(pairs)} pairs total.")

    # 2) Prepare model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model  = SentenceTransformer(MODEL_NAME, device=device).half()

    # 3) Embed each table exactly once
    table_cols = {}
    table_embs = {}
    for t in tqdm(tables, desc="Embedding tables"):
        path = os.path.join(DATA_DIR, t)
        cols, embs = embed_table(path, model, device)
        table_cols[t] = cols
        table_embs[t] = embs

    # 4) Compute all column-pair scores
    rows = []
    for t1, t2 in tqdm(pairs, desc="Scoring pairs"):
        cols1, e1 = table_cols[t1], table_embs[t1]  # [m, dim]
        cols2, e2 = table_cols[t2], table_embs[t2]  # [n, dim]

        sims = util.pytorch_cos_sim(e1, e2)  # [m,n]
        m, n = sims.shape
        flat = sims.reshape(-1)

        for idx, score in enumerate(flat):
            i, j = divmod(idx, n)
            rows.append((t1, cols1[i], t2, cols2[j], score.item()))

        # free GPU memory for this matrix
        del sims

    # 5) Write locally
    df_out = pd.DataFrame(rows, columns=[
        "table1","col1","table2","col2","join_score"
    ])
    df_out.to_csv(LOCAL_OUT, index=False)
    print(f"[+] Wrote {len(rows)} rows to {LOCAL_OUT}")

    # 6) Move to Drive
    shutil.move(LOCAL_OUT, DRIVE_OUT)
    print(f"[+] Moved result to {DRIVE_OUT}")

if __name__=="__main__":
    main()

Will process 7884 distinct tables, 12389 pairs total.


Scoring pairs: 100%|██████████| 12389/12389 [00:04<00:00, 2614.94it/s]


[+] Wrote 99917 rows to /content/join_scores_wiki_jaccard.csv
[+] Moved result to /content/drive/MyDrive/NLP/join-scores/wiki-jaccard.csv


In [15]:
#!/usr/bin/env python3

# ── CONFIGURATION ───────────────────────────────────────────────────────────
DATA_DIR   = "/content/drive/MyDrive/NLP/original-tables/wiki-jaccard/tables"
PERT_DIR   = "/content/drive/MyDrive/NLP/perturbed-tables/wiki-jaccard"
CPU_CORES  = 4                  # adjust to your VM’s core count
# perturbation rates…
TYPO_RATE  = 0.1
NULL_RATE  = 0.05
DUP_RATE   = 0.1
ABRV_RATE  = 0.5
SPLIT_RATE = 0.3
SYNONYMS   = {"first_name":"fname","last_name":"lname","date_of_birth":"dob"}
random.seed(42)
os.makedirs(PERT_DIR, exist_ok=True)

# ---- Value-level noise funcs ----
def random_insertion(s):
    i = random.randrange(len(s)+1); return s[:i]+random.choice(string.ascii_letters)+s[i:]
def random_deletion(s):
    if len(s)<=1: return s
    i = random.randrange(len(s)); return s[:i]+s[i+1:]
def random_swap(s):
    if len(s)<2: return s
    i=random.randrange(len(s)-1); L=list(s); L[i],L[i+1]=L[i+1],L[i]; return "".join(L)
def random_replacement(s):
    if not s: return s
    i=random.randrange(len(s)); return s[:i]+random.choice(string.ascii_letters)+s[i+1:]
TYPO_FUNCS = [random_insertion, random_deletion, random_swap, random_replacement]

def perturb_cell(val):
    if pd.isna(val): return val
    if isinstance(val,str) and random.random()<TYPO_RATE:
        val = random.choice(TYPO_FUNCS)(val)
    if random.random()<NULL_RATE:
        return ""
    return val

# ---- Schema funcs ----
def abbreviate(name):
    parts = re.split(r'[^A-Za-z0-9]+', name)
    return "".join(p[0] for p in parts if p).lower()

def rename_headers(headers):
    out = {}
    for h in headers:
        if h in SYNONYMS and random.random()<ABRV_RATE:
            out[h]=SYNONYMS[h]
        elif random.random()<ABRV_RATE:
            out[h]=abbreviate(h)
        else:
            out[h]=h
    return out

def split_composite_columns(df):
    candidates = [c for c in df.columns if " " in c]
    k = int(len(candidates)*SPLIT_RATE)
    for c in random.sample(candidates, k):
        parts = df[c].astype(str).str.split(" ",1,expand=True)
        df[f"{c}_1"]=parts[0]; df[f"{c}_2"]=parts[1].fillna("")
        df.drop(columns=[c],inplace=True)
    return df

def reorder_columns(df):
    cols = df.columns.tolist(); random.shuffle(cols); return df[cols]

def duplicate_rows(df):
    n = int(len(df)*DUP_RATE)
    return pd.concat([df, df.sample(n, replace=False)], ignore_index=True) if n>0 else df

def convert_numbers_to_words_df(df, rate=0.8, seed=42):
    df = df.copy(); rng = pd.api.types
    int_cols = [c for c in df.columns if (
        rng.is_integer_dtype(df[c]) or
        (rng.is_float_dtype(df[c]) and df[c].dropna().apply(float.is_integer).all())
    )]
    for col in int_cols:
        idx = df[col].dropna().sample(frac=rate, random_state=seed).index
        for i in idx:
            try: df.at[i,col]=num2words(int(df.at[i,col]))
            except: pass
        df[col]=df[col].astype(object)
    return df

# ---- Single‐file worker ----
def process_csv(fname):
    inp  = os.path.join(DATA_DIR, fname)
    outp = os.path.join(PERT_DIR, fname)
    df = pd.read_csv(inp, dtype=str).fillna("")

    # 1. Value noise (column‐wise map is slightly faster than applymap)
    for c in df.columns:
        df[c] = df[c].map(perturb_cell)

    # 2. Duplicate, rename, reorder, split
    df = duplicate_rows(df)
    df.rename(columns=rename_headers(df.columns), inplace=True)
    df = reorder_columns(df)
    df = split_composite_columns(df)

    # 3. Numeric→words
    df = convert_numbers_to_words_df(df)

    # 4. Save
    df.to_csv(outp, index=False)
    return fname

# ---- Main: parallelize over CSVs ----
if __name__=="__main__":
    files = [f for f in os.listdir(DATA_DIR) if f.endswith(".csv")]
    print(f"Found {len(files)} CSVs; perturbing with {CPU_CORES} workers…")
    with ProcessPoolExecutor(max_workers=CPU_CORES) as exe:
        for _ in tqdm(exe.map(process_csv, files), total=len(files), desc="Overall"):
            pass
    print("All done!")

Found 8489 CSVs; perturbing with 4 workers…


Overall: 100%|██████████| 8489/8489 [05:00<00:00, 28.22it/s]


All done!


In [16]:
# ── CONFIGURATION ────────────────────────────────────────────────────────────
DATA_DIR    = "/content/drive/MyDrive/NLP/perturbed-tables/wiki-jaccard"
LABELS_JSON = "/content/drive/MyDrive/NLP/original-tables/wiki-jaccard/labels.json"
MODEL_NAME  = "all-mpnet-base-v2"
LOCAL_OUT   = "/content/join_scores_wiki_jaccard.csv"   # write here first
DRIVE_OUT   = "/content/drive/MyDrive/NLP/join-scores/wiki-jaccard-perturbed.csv"
BATCH_SIZE  = 128  # increase if GPU memory allows
# ───────────────────────────────────────────────────────────────────────────────

def load_pairs(json_path):
    with open(json_path) as f:
        data = json.load(f)["train"]
    # extract unique (table1, table2) filename pairs
    return list({(r["table1"]["filename"], r["table2"]["filename"]) for r in data})

def embed_table(path, model, device):
    """Read a CSV once, build all column-texts, batch-encode, and return (cols, embs)."""
    df = pd.read_csv(path, dtype=str)  # single read
    cols = df.columns.tolist()
    texts = [analyze_column_values(df, c) for c in cols]
    del df  # free memory
    embs = model.encode(
        texts,
        convert_to_tensor=True,
        device=device,
        batch_size=BATCH_SIZE,
        show_progress_bar=False
    )
    return cols, embs

def main():
    # 1) Load table-pairs from labels.json
    pairs = load_pairs(LABELS_JSON)
    tables = sorted({t for pair in pairs for t in pair})
    print(f"Will process {len(tables)} distinct tables, {len(pairs)} pairs total.")

    # 2) Prepare model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model  = SentenceTransformer(MODEL_NAME, device=device).half()

    # 3) Embed each table exactly once
    table_cols = {}
    table_embs = {}
    for t in tqdm(tables, desc="Embedding tables"):
        path = os.path.join(DATA_DIR, t)
        cols, embs = embed_table(path, model, device)
        table_cols[t] = cols
        table_embs[t] = embs

    # 4) Compute all column-pair scores
    rows = []
    for t1, t2 in tqdm(pairs, desc="Scoring pairs"):
        cols1, e1 = table_cols[t1], table_embs[t1]  # [m, dim]
        cols2, e2 = table_cols[t2], table_embs[t2]  # [n, dim]

        sims = util.pytorch_cos_sim(e1, e2)  # [m,n]
        m, n = sims.shape
        flat = sims.reshape(-1)

        for idx, score in enumerate(flat):
            i, j = divmod(idx, n)
            rows.append((t1, cols1[i], t2, cols2[j], score.item()))

        # free GPU memory for this matrix
        del sims

    # 5) Write locally
    df_out = pd.DataFrame(rows, columns=[
        "table1","col1","table2","col2","join_score"
    ])
    df_out.to_csv(LOCAL_OUT, index=False)
    print(f"[+] Wrote {len(rows)} rows to {LOCAL_OUT}")

    # 6) Move to Drive
    shutil.move(LOCAL_OUT, DRIVE_OUT)
    print(f"[+] Moved result to {DRIVE_OUT}")

if __name__=="__main__":
    main()

Will process 7884 distinct tables, 12389 pairs total.


Scoring pairs: 100%|██████████| 12389/12389 [00:05<00:00, 2080.40it/s]


[+] Wrote 260675 rows to /content/join_scores_wiki_jaccard.csv
[+] Moved result to /content/drive/MyDrive/NLP/join-scores/wiki-jaccard-perturbed.csv


In [18]:
# 1) Load gold and perturbed scores
gold = pd.read_csv("/content/drive/MyDrive/NLP/join-scores/wiki-jaccard.csv")
pert = pd.read_csv("/content/drive/MyDrive/NLP/join-scores/wiki-jaccard-perturbed.csv")

# 2) Merge on table/column identifiers
df = (gold
      .merge(pert, on=["table1","col1","table2","col2"],
             suffixes=("_gold","_pert")))

# 3) Compute difference metrics
df["delta"]       = df["join_score_pert"] - df["join_score_gold"]
df["abs_delta"]   = df["delta"].abs()

# 4) Summary statistics
summary = {
    "mean_delta":     df["delta"].mean(),
    "mean_abs_delta": df["abs_delta"].mean(),
    "max_abs_delta":  df["abs_delta"].max(),
    "num_changed":    (df["abs_delta"] > 0).sum(),
    "total_pairs":    len(df)
}
print(summary)

# 5) (Optional) Sort by biggest changes
top_changes = df.sort_values("abs_delta", ascending=False).head(20)
print(top_changes)

{'mean_delta': np.float64(0.026456613731250783), 'mean_abs_delta': np.float64(0.041008747257167695), 'max_abs_delta': 0.3973388671875, 'num_changed': np.int64(25430), 'total_pairs': 25718}
                 table1  col1            table2  col2  join_score_gold  \
3351   HD9OLJS8ATA3.csv  col0  RS4XD8J9EP5J.csv  col0         0.161255   
2398   U8O6P8YN2K8K.csv  col3  0372NLZGK68Q.csv  col1         0.337891   
5782   7DA4PCURSY6B.csv  col1  CP01QJYBZLXM.csv  col0         0.183105   
8442   ZRK0YN7LA6RE.csv  col0  NBOTSA7FFMZH.csv  col2         0.433594   
21496  KLY8QYGL5Y84.csv  col1  7Q2UOITFNN1U.csv  col1         0.459473   
23548  DQBSUQ1E8JAG.csv  col0  KIG7OTFNAHPV.csv  col2         0.313232   
3199   OPUHNEXANP6T.csv  col0  NBOTSA7FFMZH.csv  col2         0.425049   
17950  U8O6P8YN2K8K.csv  col3  LS1FD31NDF2W.csv  col1         0.347900   
11901  ZEBBWP4RC77B.csv  col2  KVX5NY2O0SXZ.csv  col0         0.308594   
8841   JXIXDI3SY5A2.csv  col2  BUFVSABPBB4D.csv  col1         0.449707 